In [15]:
import inspect,sys,pprint
import apache_beam as beam

print pprint.pprint(inspect.getmembers(sys.modules['apache_beam.io']))


[('BigQueryDisposition',
  <class 'apache_beam.io.gcp.bigquery.BigQueryDisposition'>),
 ('BigQuerySink', <class 'apache_beam.io.gcp.bigquery.BigQuerySink'>),
 ('BigQuerySource', <class 'apache_beam.io.gcp.bigquery.BigQuerySource'>),
 ('FileBasedSink', <class 'apache_beam.io.filebasedsink.FileBasedSink'>),
 ('LexicographicKeyRangeTracker',
  <class 'apache_beam.io.range_trackers.LexicographicKeyRangeTracker'>),
 ('OffsetRangeTracker',
  <class 'apache_beam.io.range_trackers.OffsetRangeTracker'>),
 ('OrderedPositionRangeTracker',
  <class 'apache_beam.io.range_trackers.OrderedPositionRangeTracker'>),
 ('PubSubSink', <class 'apache_beam.io.gcp.pubsub.PubSubSink'>),
 ('PubSubSource', <class 'apache_beam.io.gcp.pubsub.PubSubSource'>),
 ('Read', <class 'apache_beam.io.iobase.Read'>),
 ('ReadFromAvro', <class 'apache_beam.io.avroio.ReadFromAvro'>),
 ('ReadFromTFRecord', <class 'apache_beam.io.tfrecordio.ReadFromTFRecord'>),
 ('ReadFromText', <class 'apache_beam.io.textio.ReadFromText'>),
 ('S

In [15]:
import apache_beam as beam
import re

class WordExtractingDoFn(beam.DoFn):
  """Parse each line of input text into words."""

  def __init__(self):
    super(WordExtractingDoFn, self).__init__()
    self.words_counter = beam.metrics.Metrics.counter(self.__class__, 'words')
    self.word_lengths_counter = beam.metrics.Metrics.counter(self.__class__, 'word_lengths')
    self.word_lengths_dist = beam.metrics.Metrics.distribution(
        self.__class__, 'word_len_dist')
    self.empty_line_counter = beam.metrics.Metrics.counter(self.__class__, 'empty_lines')

  def process(self, element):
    """Returns an iterator over the words of this element.
    The element is a line of text.  If the line is blank, note that, too.
    Args:
      element: the element being processed
    Returns:
      The processed element.
    """
    text_line = element.strip()
    if not text_line:
      self.empty_line_counter.inc(1)
    words = re.findall(r'[A-Za-z\']+', text_line)
    for w in words:
      self.words_counter.inc()
      self.word_lengths_counter.inc(len(w))
      self.word_lengths_dist.update(len(w))
    return words


options = {
    'job_name': 'test',
    'project': 'doit-playground',
    'teardown_policy': 'TEARDOWN_ALWAYS',
    'no_save_main_session': True
  }
opts = beam.pipeline.PipelineOptions(flags=[], **options)
p = beam.Pipeline('DirectRunner', options=opts)
lines = p | 'read' >> beam.io.ReadFromText('./input.txt')
counts = (lines
          | 'split' >> (beam.ParDo(WordExtractingDoFn())
                        .with_output_types(unicode))
          | 'pair_with_one' >> beam.Map(lambda x: (x, 1))
          | 'group' >> beam.GroupByKey()
          | 'count' >> beam.Map(lambda (word, ones): (sum(ones),word))
#           | 'sort' >> beam.OrderByKey()
          | 'map' >> beam.Map(lambda (count,word): (word, count))
         
         )
output = counts | 'format' >> beam.Map(lambda (word, c): '%s: %s' % (word, c))
output | 'write' >> beam.io.WriteToText('./output','.txt')
p.run().wait_until_finish()




'DONE'

In [7]:
%%bq query 
SELECT edgeIP 
FROM `doit-playground.playground.akamai_errors` 
LIMIT 100

edgeIP
210.158.145.71
173.197.194.176
104.78.186.101
72.246.196.175
23.212.50.113
72.246.196.175
65.202.58.161
104.78.186.101
204.93.47.231
202.239.172.93


/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-06-20 17:56:01.541739. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
